# Part 3 : Interactive Multimodal RAG

**[EN]** This notebook implements a complete, interactive multimodal RAG demo. It uses Colpali for visual search and automatically detects available LLM backends (Ollama or AWS Bedrock) based on which `.env` files exist in your workspace.
**[KR]** 이 Notebook은 완전한 대화형 멀티모달 RAG 데모를 구현합니다. Colpali를 사용하여 시각적 검색을 수행하고, 워크스페이스에 존재하는 `.env` 파일에 따라 사용 가능한 LLM 백엔드(Ollama 또는 AWS Bedrock)를 자동으로 감지합니다.

### Step 1: Environment Setup and Library Installation

**[EN]** Install the necessary libraries, including boto3 for AWS and requests for Ollama.
**[KR]** AWS용 boto3와 Ollama용 requests를 포함하여, RAG 파이프라인에 필요한 라이브러리를 설치합니다.

In [ ]:
!pip install -q "git+https://github.com/illuin-tech/colpali.git"
!pip install -q elasticsearch python-dotenv Pillow "transformers>=4.41.0" accelerate numpy torch ipywidgets requests boto3
!pip install --upgrade jupyter jupyterlab ipywidgets tqdm

### Step 2: Load Credentials and Configure Connections

**[EN]** Load environment variables from `elastic.env` and auto-detect LLM backends based on the existence of `ollama.env` and `aws.env` files.<br>
**[KR]** `elastic.env`에서 환경 변수를 로드하고, `ollama.env`와 `aws.env` 파일의 존재 여부에 따라 LLM 백엔드를 자동으로 감지합니다.

In [ ]:
from dotenv import load_dotenv
import os
from elasticsearch import Elasticsearch
import requests

# Load Elasticsearch environment variables
load_dotenv(dotenv_path='elastic.env')

ES_URL = os.getenv("ES_URL")
ES_API_KEY = os.getenv("ES_API_KEY")
if not ES_URL or not ES_API_KEY:
    raise ValueError("Please set ES_URL and ES_API_KEY in elastic.env")

# Create the Elasticsearch client
if ':' in ES_URL and not ES_URL.startswith('http'):
    es = Elasticsearch(cloud_id=ES_URL, api_key=ES_API_KEY, request_timeout=30, verify_certs=False, ssl_show_warn=False)
else:
    es = Elasticsearch(hosts=[ES_URL], api_key=ES_API_KEY, request_timeout=30, verify_certs=False, ssl_show_warn=False)
print(f"✅ Connected to Elasticsearch version: {es.info()['version']['number']}")

# --- Auto-detect available LLM backends ---
AVAILABLE_LLM_BACKENDS = []

# Check for OpenAI Compatible API (supports OpenAI, Ollama, and other compatible providers)
OPENAI_BASE_URL = None
OPENAI_API_KEY = None
OPENAI_MODEL = None
if os.path.exists('openai.env'):
    load_dotenv(dotenv_path='openai.env', override=True)
    OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")  # API key is optional
    OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
    
    try:
        # Test connection by listing models (API key is optional)
        headers = {"Authorization": f"Bearer {OPENAI_API_KEY}"} if OPENAI_API_KEY else {}
        openai_test = requests.get(f"{OPENAI_BASE_URL}/models", headers=headers, timeout=10)
        if openai_test.status_code == 200:
            print(f"✅ OpenAI Compatible API detected at: {OPENAI_BASE_URL}")
            print(f"   Using model: {OPENAI_MODEL}")
            AVAILABLE_LLM_BACKENDS.append("OpenAI")
        else:
            # Some providers don't support /models endpoint, try anyway
            print(f"✅ OpenAI Compatible API configured at: {OPENAI_BASE_URL}")
            print(f"   Using model: {OPENAI_MODEL}")
            AVAILABLE_LLM_BACKENDS.append("OpenAI")
    except requests.exceptions.RequestException as e:
        print(f"⚠️  openai.env exists but cannot connect to {OPENAI_BASE_URL}: {e}")
else:
    print("ℹ️  openai.env not found - OpenAI/Ollama backend disabled")

# Check for AWS Bedrock
BEDROCK_CLIENT = None
BEDROCK_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
if os.path.exists('aws.env'):
    load_dotenv(dotenv_path='aws.env', override=True)
    AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY", "")
    AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY", "")
    AWS_REGION = os.getenv("AWS_REGION", "ap-northeast-2")
    
    if AWS_ACCESS_KEY and AWS_SECRET_KEY and AWS_ACCESS_KEY != "<your-aws-access-key>":
        try:
            import boto3
            BEDROCK_CLIENT = boto3.client(
                "bedrock-runtime",
                aws_access_key_id=AWS_ACCESS_KEY,
                aws_secret_access_key=AWS_SECRET_KEY,
                region_name=AWS_REGION,
            )
            print(f"✅ AWS Bedrock detected in region: {AWS_REGION}")
            AVAILABLE_LLM_BACKENDS.append("AWS Bedrock")
        except Exception as e:
            print(f"⚠️  aws.env exists but cannot connect to Bedrock: {e}")
    else:
        print("⚠️  aws.env exists but credentials are not set properly")
else:
    print("ℹ️  aws.env not found - AWS Bedrock backend disabled")

# Summary
if not AVAILABLE_LLM_BACKENDS:
    print("\n❌ No LLM backends available! Please create openai.env or aws.env")
else:
    print(f"\n🎉 Available LLM backends: {AVAILABLE_LLM_BACKENDS}")

### Step 3: Define Helpers and Load the Embedding Model

**[EN]** Define helper functions for visualization, vector generation, and LLM calls (supporting both Ollama and Bedrock). Load the `ColQwen` model for search.<br>
**[KR]** 시각화, 벡터 생성, LLM 호출(Ollama와 Bedrock 모두 지원)을 위한 헬퍼 함수를 정의합니다. 검색을 위해 `ColQwen` 모델을 로드합니다.

In [ ]:
import base64
from IPython.display import display, HTML
import torch
import numpy as np
import json
import time
from colpali_engine.models import ColQwen3, ColQwen3Processor
from transformers import AutoConfig

SYSTEM_PROMPT = """You are an intelligent document analysis assistant. Your task is to analyze a document image that a search system has retrieved in response to a user's query. Your response should be structured as follows:

1.  **Relevance Assessment**: Start by explaining how relevant the document is to the user's query.
2.  **Summary**: Provide a concise summary of the document's key information.
3.  **Direct Answer**: Directly answer the user's query based on the document's content.
4.  **Contextual Explanation**: If the document is not a perfect match for the query, explain why it was likely the most relevant result found."""

def display_results(hits):
    """Renders search results as an HTML table."""
    if not hits:
        print("No documents found.")
        return
    html = "<div style='display:flex; flex-wrap:wrap;'>"
    for i, hit in enumerate(hits):
        doc_id = hit["_id"]
        score = hit["_score"]
        path = hit["_source"].get("image_path", "")
        category = hit["_source"].get("category", "N/A")
        try:
            with open(path, "rb") as image_file:
                img_str = base64.b64encode(image_file.read()).decode()
                html += f"<div style='margin:10px; padding:10px; border:1px solid #ddd; text-align:center; width: 220px;'><b>Rank #{i+1}</b><br><img src='data:image/png;base64,{img_str}' style='width:200px; height:auto; margin-top:5px;'><br><div style='font-size:12px; margin-top:5px;'><b>ID:</b> {doc_id[:15]}...<br><b>Score:</b> {score:.4f}<br><b>Category:</b> {category}</div></div>"
        except Exception:
            html += f"<div style='margin:10px; padding:10px; border:1px solid #ddd; text-align:center; width: 220px; height: 300px;'><b>Rank #{i+1}</b><br><div style='width:200px; height:200px; background-color:#f0f0f0; margin-top:5px; display:flex; align-items:center; justify-content:center; font-size:12px;'>Image not available</div><div style='font-size:12px; margin-top:5px;'><b>ID:</b> {doc_id[:15]}...<br><b>Score:</b> {score:.4f}<br><b>Category:</b> {category}</div></div>"
    html += "</div>"
    display(HTML(html))

def get_media_type(image_path):
    """Determines the media type from the file extension."""
    ext = os.path.splitext(image_path)[1].lower()
    return {".jpeg": "image/jpeg", ".jpg": "image/jpeg", ".png": "image/png", ".webp": "image/webp"}.get(ext, "image/jpeg")

def generate_with_openai(query_text, image_base64, image_path):
    """Generate answer using OpenAI Compatible API."""
    media_type = get_media_type(image_path)
    
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    
    openai_request = {
        "model": OPENAI_MODEL,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:{media_type};base64,{image_base64}"
                        }
                    },
                    {
                        "type": "text",
                        "text": f"User Query: '{query_text}'\n\nPlease analyze the provided document image."
                    }
                ]
            }
        ],
        "max_tokens": 2048
    }
    
    response = requests.post(
        f"{OPENAI_BASE_URL}/chat/completions",
        headers=headers,
        json=openai_request,
        timeout=120
    )
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

def generate_with_bedrock(query_text, image_base64, image_path):
    """Generate answer using AWS Bedrock."""
    bedrock_request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2048,
        "system": SYSTEM_PROMPT,
        "messages": [{
            "role": "user",
            "content": [
                {"type": "image", "source": {"type": "base64", "media_type": get_media_type(image_path), "data": image_base64}},
                {"type": "text", "text": f"User Query: '{query_text}'\n\nPlease analyze the provided document image."}
            ]
        }]
    }
    response = BEDROCK_CLIENT.invoke_model(
        body=json.dumps(bedrock_request_body),
        modelId=BEDROCK_MODEL_ID,
        contentType="application/json",
        accept="application/json"
    )
    response_body = json.loads(response.get("body").read())
    return response_body.get("content", [{}])[0].get("text", "No response generated.")

def generate_llm_answer_with_image(query_text, hits, llm_backend):
    """Encodes the top image and generates an answer using selected LLM backend."""
    if not hits:
        print("No documents found, cannot generate answer.")
        return
    top_hit = hits[0]
    image_path = top_hit["_source"].get("image_path")
    if not image_path or not os.path.exists(image_path):
        print("Top result has no valid image path.")
        return
    
    print(f"\n🔍 Analyzing top image with {llm_backend}: {image_path}")
    
    with open(image_path, "rb") as image_file:
        image_base64 = base64.b64encode(image_file.read()).decode('utf-8')
    
    try:
        if llm_backend == "OpenAI":
            result = generate_with_openai(query_text, image_base64, image_path)
        elif llm_backend == "AWS Bedrock":
            result = generate_with_bedrock(query_text, image_base64, image_path)
        else:
            result = "Unknown LLM backend selected."
        print(f"\n[FINAL LLM RESPONSE]:\n{result}")
    except Exception as e:
        print(f"\n❌ Error calling {llm_backend}: {e}")

# Set up the device (GPU or CPU)
device_map = "cpu"
if torch.backends.mps.is_available(): device_map = "mps"
elif torch.cuda.is_available(): device_map = "cuda:0"
print(f"Using device: {device_map}")

# Load the ColQwen model used for indexing.
MODEL_NAME = "TomoroAI/tomoro-colqwen3-embed-4b"

# Load config first and fix rope_scaling if None
config = AutoConfig.from_pretrained(MODEL_NAME, trust_remote_code=True)
if hasattr(config, 'text_config') and config.text_config.rope_scaling is None:
    config.text_config.rope_scaling = {"mrope_section": [24, 20, 20], "type": "default"}

# Try flash_attention_2 first, fall back to sdpa if not available
attn_impl = "flash_attention_2" if device_map != "cpu" else "eager"
try:
    import flash_attn
except ImportError:
    attn_impl = "sdpa" if device_map != "cpu" else "eager"
    print(f"flash-attn not installed, using '{attn_impl}' attention instead.")

model = ColQwen3.from_pretrained(
    MODEL_NAME,
    config=config,
    torch_dtype=torch.bfloat16 if device_map != "cpu" else torch.float32,
    device_map=device_map,
    attn_implementation=attn_impl,
    trust_remote_code=True
).eval()

processor = ColQwen3Processor.from_pretrained(MODEL_NAME)
print(f"Embedding model '{MODEL_NAME}' loaded successfully.")

def create_colqwen_query_vectors(query_text, model, processor):
    """Creates multi-vector embeddings for a text query."""
    inputs = processor.process_queries([query_text]).to(model.device)
    with torch.no_grad(): outputs = model(**inputs)
    return outputs.cpu().to(torch.float32).numpy().tolist()[0]

def to_avg_vector(vectors):
    """Calculates a single, normalized average vector."""
    vectors_array = np.array(vectors)
    avg_vector = np.mean(vectors_array, axis=0)
    norm = np.linalg.norm(avg_vector)
    return (avg_vector / norm).tolist() if norm > 0 else avg_vector.tolist()

### Step 4: Create and Display the Interactive RAG Chatbot

**[EN]** This cell creates an interactive UI using `ipywidgets`. You can select a search strategy, input a query, and trigger the full multimodal RAG pipeline.<br>
**[KR]** 이 셀은 `ipywidgets`를 사용하여 대화형 UI를 생성합니다. 검색 전략을 선택하고, 쿼리를 입력하여 전체 멀티모달 RAG 파이프라인을 실행할 수 있습니다.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# --- 1. Define Constants and Main RAG Pipeline Function ---
SEARCH_MODES = {
    "A. Colpali(colqwen) RAG search (Part 1)": {
        "index": "colqwen3-rvlcdip-demo-part1",
        "multi_vector_field": "colqwen_vectors"
    },
    "B. Average RAG search (Part 2 - KNN Only)": {
        "index": "colqwen3-rvlcdip-demo-part2",
        "avg_vector_field": "colqwen_avg_vector"
    },
    "C. Rescore RAG search (Part 2 - KNN + Rescore)": {
        "index": "colqwen3-rvlcdip-demo-part2",
        "avg_vector_field": "colqwen_avg_vector",
        "multi_vector_field": "colqwen_vectors"
    }
}

def run_rag_pipeline(query_text, search_mode, llm_backend):
    """Main RAG pipeline function."""
    mode_config = SEARCH_MODES[search_mode]
    index_name = mode_config["index"]
    es_query_body = None

    print(f"--- Running RAG for query: '{query_text}' ---")
    print(f"    Search Mode: {search_mode}")
    print(f"    LLM Backend: {llm_backend}")
    
    try:
        start_time = time.time()
        query_multi_vectors = create_colqwen_query_vectors(query_text, model, processor)
        if not query_multi_vectors: raise ValueError("Failed to create query vectors.")

        if search_mode.startswith("A."):
            es_query_body = {"size": 10, "query": {"script_score": {"query": {"match_all": {}}, "script": {"source": f"maxSimDotProduct(params.query_vector, '{mode_config['multi_vector_field']}')", "params": {"query_vector": query_multi_vectors}}}}, "_source": ["image_path", "category"]}
        elif search_mode.startswith("B."):
            query_avg_vector = to_avg_vector(query_multi_vectors)
            es_query_body = {"size": 10, "knn": {"field": mode_config['avg_vector_field'], "query_vector": query_avg_vector, "k": 200, "num_candidates": 500}, "_source": ["image_path", "category"]}
        else:
            query_avg_vector = to_avg_vector(query_multi_vectors)
            knn_query = {"field": mode_config['avg_vector_field'], "query_vector": query_avg_vector, "k": 200, "num_candidates": 500}
            rescore_definition = {"window_size": 50, "query": {"rescore_query": {"script_score": {"query": {"match_all": {}}, "script": {"source": f"maxSimDotProduct(params.query_vector, '{mode_config['multi_vector_field']}')", "params": {"query_vector": query_multi_vectors}}}}, "query_weight": 0.0, "rescore_query_weight": 1.0}}
            es_query_body = {"size": 10, "knn": knn_query, "rescore": rescore_definition, "_source": ["image_path", "category"]}

        response = es.search(index=index_name, body=es_query_body)
        end_time = time.time()
        latency_ms = (end_time - start_time) * 1000
        hits = response["hits"]["hits"]

        print(f"\n[VISUAL SEARCH RESULTS] - Retrieved {len(hits)} documents in {latency_ms:.2f} ms:")
        display_results(hits)
        
        # Generate answer from the top image using selected LLM backend
        generate_llm_answer_with_image(query_text, hits, llm_backend)

    except Exception as e:
        print(f"\nAn error occurred: {e}")

# --- 2. Create and Display the UI Widgets ---
query_input = widgets.Text(placeholder='Enter your query here and press Enter', description='Query:', layout=widgets.Layout(width='95%'))
search_mode_selector = widgets.RadioButtons(options=list(SEARCH_MODES.keys()), description='Search Mode:', disabled=False, layout=widgets.Layout(width='max-content'))

# LLM Backend selector - only show available backends
if AVAILABLE_LLM_BACKENDS:
    llm_backend_selector = widgets.RadioButtons(
        options=AVAILABLE_LLM_BACKENDS,
        value=AVAILABLE_LLM_BACKENDS[0],
        description='LLM Backend:',
        disabled=False,
        layout=widgets.Layout(width='max-content')
    )
else:
    llm_backend_selector = widgets.HTML(value="<b style='color:red;'>No LLM backend available. Please create ollama.env or aws.env</b>")

output_area = widgets.Output()
example_queries = ["Do you have a benefits policy change notice from HR?", "HR에서 보내온 복리후생 정책 변경 안내문이 있나?"]
example_buttons = [widgets.Button(description=q, layout=widgets.Layout(width='auto')) for q in example_queries]
button_box = widgets.HBox([widgets.Label("Examples:")] + example_buttons)

def on_search_triggered(query):
    if not query: 
        with output_area: output_area.clear_output(); print("Please enter a query.")
        return
    if not AVAILABLE_LLM_BACKENDS:
        with output_area: output_area.clear_output(); print("❌ No LLM backend available!")
        return
    with output_area: 
        output_area.clear_output(wait=True)
        run_rag_pipeline(query, search_mode_selector.value, llm_backend_selector.value)

def handle_text_submit(sender): on_search_triggered(sender.value)
def handle_button_click(button): query_input.value = button.description; on_search_triggered(button.description)

query_input.on_submit(handle_text_submit)
for btn in example_buttons: btn.on_click(handle_button_click)

# Create UI layout with LLM backend selector
selector_box = widgets.HBox([search_mode_selector, widgets.HTML(value="&nbsp;&nbsp;&nbsp;"), llm_backend_selector])
ui = widgets.VBox([selector_box, button_box, query_input, output_area])
display(ui)

### Step 5: Clean Up Memory (Optional)

**[EN]** As a best practice, explicitly delete the model and processor to free up GPU or system memory after the demonstration is complete.<br>
**[KR]** 모범 사례로서, 데모가 완료된 후 모델과 프로세서를 명시적으로 삭제하여 GPU 또는 시스템 메모리를 확보합니다.

In [ ]:
# import gc

# try:
#     del model
#     del processor
#     print("Model and processor variables deleted.")
# except NameError:
#     print("Model and processor variables not found, skipping deletion.")

# if 'torch' in locals() and torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print("CUDA cache cleared.")
# elif 'torch' in locals() and torch.backends.mps.is_available():
#     torch.mps.empty_cache()
#     print("MPS cache cleared.")

# # Call Python's garbage collector to clean up memory.
# gc.collect()
# print("Memory cleanup complete.")